In [23]:
import tensorflow as tf
import numpy as np
from numpy import ndarray


In [24]:
class Layer:
    def __init__(self,inputAmnt,outputAmnt):
        def sharmaAct(input):
            x = input
            mask1 = x < -0.5
            mask2 = (-0.5 <= x) & (x <= 0.5)

            result = np.zeros_like(x)  # Initialize an array of zeros
            result[mask1] = (x[mask1] + 1.5) / np.sqrt(np.e)
            result[mask2] = np.exp(x[mask2])
            result[~(mask1 | mask2)] = x[~(mask1 | mask2)] + 1
            return result
        def sharmaActPrime(input):
            x = input
            mask1 = x < -0.5
            mask2 = (-0.5 <= x) & (x <= 0.5)
            
            result = np.zeros_like(x)  # Initialize an array of zeros
            result[mask1] =  1 / np.sqrt(np.e) 
            result[mask2] = np.exp(x[mask2])
            result[~(mask1 | mask2)] = 1 
            return result
        self.f = sharmaAct
        self.fPrime = sharmaActPrime
        self.w = np.random.uniform(low=-1,high=1,size=(outputAmnt,inputAmnt))
        self.b = np.random.uniform(low=-1,high=1,size=(outputAmnt,1))
        self.linearCombo = 0
        self.input = 0
    def compute(self,input):
        self.input = input
        self.linearCombo = self.w @ input + self.b
        return self.f(self.linearCombo)
    def compPrime(self):
        return self.fPrime(self.linearCombo)

In [25]:
class inputLayer:
    def __init__(self,activation):
        def relu(input):
            return np.maximum(input,0)
        def reluPrime(input):
            return np.greater(input, 0).astype(np.float32)
        def sigmoid(input):
            return 1/(1+np.exp(-input))
        def sigmoidPrime(input):
            return -np.exp(-input)/(1+np.exp(input))
        def tanhPrime(input):
            np.power(1/(np.cosh(input)),2)
        def sharmaAct(input):
            x = input
            mask1 = x < -0.5
            mask2 = (-0.5 <= x) & (x <= 0.5)

            result = np.zeros_like(x)  # Initialize an array of zeros
            result[mask1] = (x[mask1] + 1.5) / np.sqrt(np.e)
            result[mask2] = np.exp(x[mask2])
            result[~(mask1 | mask2)] = x[~(mask1 | mask2)] + 1
            return result
        def sharmaActPrime(input):
            x = input
            mask1 = x < -0.5
            mask2 = (-0.5 <= x) & (x <= 0.5)
            
            result = np.zeros_like(x)  # Initialize an array of zeros
            result[mask1] =  1 / np.sqrt(np.e) 
            result[mask2] = np.exp(x[mask2])
            result[~(mask1 | mask2)] = 1 
            return result
        val = {"relu":[relu,reluPrime],"sigmoid":[sigmoid,sigmoidPrime],"tanh":[np.tanh,tanhPrime],'sharma':[sharmaAct,sharmaActPrime]}
        self.f = val[activation][0]
    

In [61]:
class ANN:
    def __init__(self,input: inputLayer,*layers):
        self.layers = list(layers)
        self.input = input
    def divideByThing(self,input):
        if(np.sum(np.abs(input)) == 0):
            return input
        return input/np.sum(np.abs(input))
    def predict(self,input):
        output = self.input.f(input)
        for k in self.layers:
            output = k.compute(output)
        return np.apply_along_axis(self.divideByThing,0,output)
    #all in 1 gradient descent using some stuff
    def gradientDescent(self,inputMat,outputMat):
        out = self.predict(inputMat)
        godMat = np.identity(n = len(self.layers[-1].w))
        chainRes = 2 * (out-outputMat) * self.layers[-1].compPrime()
        self.layers.reverse()
        prevW = -1
        for k in self.layers:
                if(not isinstance(prevW,int)):
                    godMat = godMat @ prevW
                    chainRes *= godMat @ k.compPrime()
                val = np.transpose(godMat) @ chainRes                 
                prevW = k.w
                k.b -= val @ np.ones(shape=(val.shape[1],1))

        self.layers.reverse()


In [5]:
x = np.array([[1,2],[2,3]])
def divideByThing(input):
        if(np.sum(np.abs(input)) == 0):
            return input
        return input/np.sum(np.abs(input))
def har(input):
      return np.apply_along_axis(divideByThing,0,input)
print(har(x))

[[0.33333333 0.4       ]
 [0.66666667 0.6       ]]


In [62]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
def encode(input):
    z = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    z[input] = 1
    return z
# Flatten the images
x_train = x_train.reshape(x_train.shape[0], -1)/255  # input for training
x_test = x_test.reshape(x_test.shape[0], -1)/255  # input for testing
y_train = y_train.reshape(1,60_000)
output_test = np.apply_along_axis(encode,0,y_train)
x_train = np.transpose(x_train)
realTrain = x_train.reshape(-1,784,1)
realYTrain = np.transpose(y_train)



In [63]:
model = ANN(inputLayer('sharma'),Layer(784,15),Layer(15,10))

In [ ]:
model.gradientDescent(x_train,output_test)
print("")

In [9]:

def percentAcc(mod):
  global realTrain,realYTrain
  error= 0
  
  for (input,output) in zip(realTrain,realYTrain):
    if(np.argmax(mod.predict(input)) != (output)):
      error += 1
  return 1-(error)/60_000
print(percentAcc(model))

In [66]:
x = np.array([1,2,3])
y = np.array([0,4,8])
np.einsum("i,j->ij",x,y)


TypeError: Field elements must be 2- or 3-tuples, got '4'

In [22]:
np.einsum("i,j->ij",y,x)


array([[ 0,  0,  0],
       [ 4,  8, 12],
       [ 8, 16, 24]])